# Introduction
This notebook was created by Garrett R. Dowdy (gdowdy3@gmail.com) to ease the process of scheduling interviews for Ashdown officer positions.

The notebook consists of a series of gray boxes ("cells") containing code.
The cells are divided up into sections by bold headers  -- "Introduction", "Setup", and so on.
Many of these headers are accompanied by explanatory text.

To run this code:

1. Verify that the "Interviewee Information.csv" and "Interviewer Information.csv" files have been uploaded to the same JuliaBox folder as the .pynb file that you clicked to open this notebook. Note: the spelling must be *exact*.
2. On the menu bar above, select **Cell** --> **All Output** --> **Clear**
3. Moving down the document, run each cell in sequence,  reading the explanatory text and checking for errors as you go.
4. Check the summary statistics printed by the final cell.
5. Check the output csv files added to the current JuliaBox folder (see previous tab in your browser).
6. If you are happy with the results, great!  If not, tweak the objective weights in the "Define the Objective Function" section and start over, returning to Step 2.
7. You can exit this notebook gracefully by selecting **File** --> **Close and Halt**.

A cell is run by clicking in the gray box and then pressing [ctrl] + [enter] on your keyboard. 
You will know that a cell is executed when the "In[ ]" to its left is replaced with "In[#]", where "#" is a number. 
"In[\*]" indicates that the cell is currently executing.  You can use the up and down arrow keys to move between cells.

In [80]:
# UPDATE LOG
# Iteration   Changes
#------------------------
# 4           Added export of the composite schedule
#             Added export of interviewee emails next to their names in the composite schedule
#
# 5           Added import of interviewee availability
#             Added interviewee availability to objective function
#             Added report of summary statistics
#
# 6           Eliminated the "exception" decision variables.
#             Reduced the number of constraints associated with the one AHEC member requirement
#             Allowed for multiple tech committee members to be assigned to a meeting
#
# 7           Updated the import of interviewee data to reflect the format of Rio's Google form.
#             Added an exception variable, allowing for AHEC members not responsible for the committee being interviewed for to show up, if needed.
#
# 8           Ensure that each AHEC member takes at least one break every two hours.
#             Ensure that each committee chair takes at least one break every two hours.
#
# 9           Uses CBC instead of Gurobi
#
# 10          Updated the code to reflect the new format of the input data for the interviewers.
#
# 11          Added a check of the feasibility of the solution returned after timeout.
#
# 12          Added commentary and made the code more presentable.
#
# 13          Slight cosmetic improvements.
#
# 14          Added Old AHEC to the mix.
#             Only one Tech committee member is allowed per interview.

# Potential Improvements
## Figure out how to quickly re-run the code.
## Figure out how to solve the problem again when someone comes back saying that they are not available.
## Implement a scheme to "break symmetry".
## Add a happy sound to indicate successful solution.

# Set Up
## Add Packages
The code makes use of several "packages".  Before using these packages we have to add them.  You don't have to do this every time you run the code.  Just do it once, every time you sit down to work with the code, to make sure the packages are up to date.

In [81]:
# # # Add packages as necessary
# Pkg.update()
# Pkg.add("JuMP")
# Pkg.add("Clp")
# Pkg.add("DataFrames")

## Define Custom Functions
The code makes use of some custom-made functions.  So that they don't disrupt the flow of logic in the body of the code below, it is best to define them here.  As with the previous cell, you don't have to run these cells every time you run the code.  Just do it once, every time you sit down to work with the code.

In [82]:
# Create the mapping between the times and the (coarse) time indices
## The "coarse" time indices refer to hours.  These hours are later subdivided into 20-minute segments

## specify the hours each day when people can specify they are available (these should be listed in chronological order)
friTimes = ["3-4pm", "4-5pm", "5-6pm", "6-7pm", "7-8pm", "8-9pm"]
satTimes = ["10-11am", "11-12pm", "12-1pm", "1-2pm", "2-3pm","3-4pm", "4-5pm", "5-6pm", "6-7pm", "7-8pm", "8-9pm"]
sunTimes = ["10-11am", "11-12pm", "12-1pm", "1-2pm", "2-3pm","3-4pm", "4-5pm", "5-6pm", "6-7pm", "7-8pm", "8-9pm"]

# count the number of coarse time periods
n_T_coarse = length(friTimes) + length(satTimes) + length(sunTimes)

function mapDayAndTimeToCoarseIndex(day::String, time::String)
    #Inputs:
    # day = "Friday", "Saturday", or "Sunday"
    # time = "3-4pm", "12-1pm", or any of the other entries in the arrays above
    #
    #Outputs:
    # t = a (coarse) time index corresponding to the specified day and time
    
    #initialize t
    t = 
    
    if day == "Friday"
        # look for the specified time in the friday array
        ind = find(friTimes .== time)
        
        # check that exactly one index was returned
        if length(ind) != 1
            error("The string $time does not appear friTimes array (or appears multiple times)")
        end
        
        # set t
        t = ind

    elseif day == "Saturday"
        # look for the specified time in the saturday array
        ind = find(satTimes .== time)
        
        # check that exactly one index was returned
        if length(ind) != 1
            error("The string $time does not appear satTimes array (or appears multiple times)")
        end
        
        # set t
        t = length(friTimes) + ind

    elseif day == "Sunday"
        # look for the specified time in the sunday array
        ind = find(satTimes .== time)
        
        # check that exactly one index was returned
        if length(ind) != 1
            error("The string $time does not appear sunTimes array (or appears multiple times)")
        end
        
        # set t
        t = length(friTimes) + length(satTimes) + ind

    else
        error("Unrecognized day")
    end
    
    #return the result
    return t[1]
    
end


mapDayAndTimeToCoarseIndex (generic function with 1 method)

In [83]:
function subdivideAvailabilityMatrix(V::Array{Integer,2})
    #Inputs:
    #   V = an M x N binary matrix indicating the availability of some group of
    #       individuals.
    #
    #Outputs:
    #   V_expanded = an M x 3N binary matrix indicating the availability of
    #       some group of individuals.  This is an expanded version of the
    #       input matrix in the sense that each column is replicated 3 times.
    
    #read the size of the input matrix
    M = size(V,1)
    N = size(V,2)
    
    #intialize the expanded matrix
    V_expanded = zeros(Integer,M,3*N)
    
    #fill out each column of the expanded matrix
    for j = 1:N
       for i = 1:3
            V_expanded[:,(j-1)*3 + i] = V[:,j]
       end    
    end
    
    return V_expanded
    
end

subdivideAvailabilityMatrix (generic function with 1 method)

# Read in the Problem Data
There are two sources of data which define the problem:

1. A csv file describing the interviewees: "Interviewee Information.csv".
2. A csv file describing the interviewers: "Interviewer Information.csv".

In this section of the code, these two data files are imported and interpreted.

Executing the following two cells displays the top and bottom portions of the interviewer information input table.  Check the output to make sure there isn't any funny business going on.

In [84]:
# Read in the AHEC data
using DataFrames, DataArrays
raw = readtable("Interviewer Information.csv")

String, String) in module Main at In[59]:21 overwritten at In[82]:21.


,Timestamp,Name,Email,Are_you_AHEC_a_Chair_or_Tech,What_Committee_do_you_represent_Tech_please_select_tech_again_,When_are_you_available_Friday_4_6_,When_are_you_available_Saturday_4_7_,When_are_you_available_Sunday_4_8_
1,4/2/2018 15:43,Nicole Moody,nmoody@mit.edu,Old AHEC,Coffee Hour,"6-7pm, 7-8pm, 8-9pm",NA,"4-5pm, 5-6pm, 6-7pm, 7-8pm"
2,4/2/2018 15:46,Sylvia Dai,sylviad@mit.edu,AHEC,Coffee Hour,"7-8pm, 8-9pm","1-2pm, 2-3pm, 3-4pm, 4-5pm, 5-6pm, 6-7pm, 7-8pm, 8-9pm","10-11am, 11-12pm, 12-1pm, 1-2pm, 2-3pm, 3-4pm"
3,4/2/2018 16:23,Haley Schilling,hschill@mit.edu,AHEC,Communities,NA,"10-11am, 11-12pm, 6-7pm, 7-8pm, 8-9pm","10-11am, 11-12pm, 12-1pm, 1-2pm, 2-3pm, 3-4pm, 4-5pm, 5-6pm, 6-7pm, 7-8pm, 8-9pm"
4,4/2/2018 17:36,Ricardo Baptista,rsb@mit.edu,Chair,Events,"7-8pm, 8-9pm","1-2pm, 2-3pm, 3-4pm, 4-5pm, 5-6pm","1-2pm, 2-3pm, 3-4pm, 4-5pm, 5-6pm"
5,4/2/2018 21:07,Eric,ealt@mit.edu,Old AHEC,Brunch,"7-8pm, 8-9pm","10-11am, 11-12pm, 12-1pm, 1-2pm, 2-3pm, 3-4pm, 4-5pm, 5-6pm, 6-7pm, 7-8pm, 8-9pm","2-3pm, 3-4pm, 4-5pm, 5-6pm, 6-7pm"
6,4/3/2018 1:47,Neha Bokil,nbokil@mit.edu,AHEC,Operations,8-9pm,"11-12pm, 12-1pm, 1-2pm, 2-3pm, 3-4pm, 4-5pm, 5-6pm, 6-7pm","11-12pm, 12-1pm, 1-2pm, 2-3pm, 3-4pm, 4-5pm, 5-6pm, 6-7pm"
7,4/3/2018 19:07,haluk,haluk@mit.edu,Chair,Communities,"3-4pm, 4-5pm, 5-6pm, 6-7pm","10-11am, 11-12pm, 12-1pm, 1-2pm, 2-3pm, 3-4pm",NA
8,4/4/2018 8:51,Mukund Gupta,guptam@mit.edu,Chair,Brunch,"7-8pm, 8-9pm",NA,"10-11am, 11-12pm, 1-2pm, 2-3pm, 3-4pm, 4-5pm"
9,4/4/2018 10:11,Liam Comidy,comidy@mit.edu,AHEC,Brunch,"3-4pm, 4-5pm, 5-6pm, 6-7pm, 7-8pm","11-12pm, 12-1pm, 1-2pm, 2-3pm","10-11am, 11-12pm, 12-1pm, 1-2pm, 2-3pm, 3-4pm, 4-5pm, 5-6pm, 6-7pm, 7-8pm, 8-9pm"
10,4/4/2018 12:47,Gary Lee,glcf411@mit.edu,Tech,Tech,"3-4pm, 4-5pm","4-5pm, 5-6pm, 6-7pm, 7-8pm, 8-9pm","10-11am, 11-12pm, 8-9pm"


In [85]:
tail(raw)

LoadError: UndefVarError: tail not defined

In [86]:
# describe the formatting of the data table
nameCol = 2
roleCol = 4
comCol = 5
firstAvailabilityCol = 3
friCol = 6
satCol = 7
sunCol = 8

# intialize the vectors to contain the rows of each group
AHEC_rows= Int[]
chair_rows = Int[]
tech_rows = Int[]
oldAHEC_rows = Int[]

# loop over the rows of the table
for r = 1:size(raw,1)
    # check for an AHEC member
    if raw[r,roleCol] == "AHEC"
        AHEC_rows = push!(AHEC_rows,r)
    elseif raw[r,roleCol] == "Chair"
        chair_rows = push!(chair_rows,r)
    elseif raw[r,roleCol] == "Tech"
        tech_rows = push!(tech_rows,r)
    elseif raw[r,roleCol] == "Old AHEC"
        oldAHEC_rows = push!(oldAHEC_rows,r)
    else
        error("The role in row $r was unrecognized.  Valid roles are AHEC, Old AHEC, Chair, and Tech, nothing else.")
    end
end

#extract the AHEC names
AHEC_names = raw[AHEC_rows,nameCol];
n_A = length(AHEC_names);  #the number of AHEC members

# extract the committee each AHEC member is associated with
AHEC_com_names = raw[AHEC_rows,comCol]

# create the dictionaries identifying the committees with particular codes
lookup_com_codes = Dict{String,Integer}("Brunch" => 1, "Coffee Hour" => 2, "Communities" => 3, "Events" => 4, "Operations" => 5)
lookup_com_names = map(reverse, lookup_com_codes)

# convert the committee names into committee codes
M_A = zeros(Integer,length(AHEC_com_names),1)
for i = 1:length(AHEC_com_names)
    M_A[i] = lookup_com_codes[AHEC_com_names[i]]
end

# extract the availability
V_A_dense = zeros(Integer,n_T_coarse,n_A);
friAvail = raw[AHEC_rows,friCol]
satAvail = raw[AHEC_rows,satCol]
sunAvail = raw[AHEC_rows,sunCol]

for a in 1:n_A
    # friday
    timesString = friAvail[a]
    if typeof(timesString) != NAtype
        for time in friTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Friday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_A_dense[t,a] = 1
            end
        end
    end
    
    # saturday
    timesString = satAvail[a]
    if typeof(timesString) != NAtype
        for time in satTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Saturday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_A_dense[t,a] = 1
            end
        end
    end
    
    # sunday
    timesString = sunAvail[a]
    if typeof(timesString) != NAtype
        for time in sunTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Sunday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_A_dense[t,a] = 1
            end
        end
    end
end

# subdivide the availability matrix
n_T = 3*n_T_coarse
V_A = subdivideAvailabilityMatrix(transpose(V_A_dense))

# transpose the availability matrix
V_A = transpose(V_A);

In [87]:
# Read in the old AHEC data

# extract the names of the old AHEC members
oldAHEC_names = raw[oldAHEC_rows,nameCol]
n_O = length(oldAHEC_names) # the number of old AHEC members

# extract the committee each old AHEC member is associated with
oldAHEC_com_names = raw[oldAHEC_rows,comCol]

# convert the committee names into committee codes
M_O = zeros(Integer,n_O,1)
for i = 1:n_O
    M_O[i] = lookup_com_codes[oldAHEC_com_names[i]]
end

# extract the availability
V_O_dense = zeros(Integer,n_T_coarse,n_O);
friAvail = raw[oldAHEC_rows,friCol]
satAvail = raw[oldAHEC_rows,satCol]
sunAvail = raw[oldAHEC_rows,sunCol]

for o in 1:n_O
    # friday
    timesString = friAvail[o]
    if typeof(timesString) != NAtype
        for time in friTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Friday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_O_dense[t,o] = 1
            end
        end
    end
    
    # saturday
    timesString = satAvail[o]
    if typeof(timesString) != NAtype
        for time in satTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Saturday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_O_dense[t,o] = 1
            end
        end
    end
    
    # sunday
    timesString = sunAvail[o]
    if typeof(timesString) != NAtype
        for time in sunTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Sunday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_O_dense[t,o] = 1
            end
        end
    end
end

# subdivide the availability matrix
n_T = 3*n_T_coarse
V_O = subdivideAvailabilityMatrix(transpose(V_O_dense))

#transpose the availability matrix
V_O = transpose(V_O);

In [88]:
# Read in the committee chair data

# extract the names of the Chairs
chair_names = raw[chair_rows,nameCol]
n_C = length(chair_names) # the number of committee Chairs

# extract the committee each Chair is associated with
chair_com_names = raw[chair_rows,comCol]

# convert the committee names into committee codes
M_C = zeros(Integer,n_C,1)
for i = 1:n_C
    M_C[i] = lookup_com_codes[chair_com_names[i]]
end

# extract the availability
V_C_dense = zeros(Integer,n_T_coarse,n_C);
friAvail = raw[chair_rows,friCol]
satAvail = raw[chair_rows,satCol]
sunAvail = raw[chair_rows,sunCol]

for c in 1:n_C
    # friday
    timesString = friAvail[c]
    if typeof(timesString) != NAtype
        for time in friTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Friday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_C_dense[t,c] = 1
            end
        end
    end
    
    # saturday
    timesString = satAvail[c]
    if typeof(timesString) != NAtype
        for time in satTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Saturday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_C_dense[t,c] = 1
            end
        end
    end
    
    # sunday
    timesString = sunAvail[c]
    if typeof(timesString) != NAtype
        for time in sunTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Sunday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_C_dense[t,c] = 1
            end
        end
    end
end

# subdivide the availability matrix
n_T = 3*n_T_coarse
V_C = subdivideAvailabilityMatrix(transpose(V_C_dense))

#transpose the availability matrix
V_C = transpose(V_C);

In [89]:
# Read in the Tech Committee availability

# extract the names of the committee members
tech_names = raw[tech_rows,nameCol]
n_E = length(tech_names) #the number of committee Chairs

# extract the availability
V_E_dense = zeros(Integer,n_T_coarse,n_E);
friAvail = raw[tech_rows,friCol]
satAvail = raw[tech_rows,satCol]
sunAvail = raw[tech_rows,sunCol]

for e in 1:n_E
    # friday
    timesString = friAvail[e]
    if typeof(timesString) != NAtype
        for time in friTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Friday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_E_dense[t,e] = 1
            end
        end
    end
    
    # saturday
    timesString = satAvail[e]
    if typeof(timesString) != NAtype
        for time in satTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Saturday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_E_dense[t,e] = 1
            end
        end
    end
    
    # sunday
    timesString = sunAvail[e]
    if typeof(timesString) != NAtype
        for time in sunTimes
            if contains(timesString,time) # this interviewee indicated that they are available during the current time
                # map the current time string to the corresponding (coarse) time index
                t = mapDayAndTimeToCoarseIndex("Sunday",time)

                # populate the corresponding entry of the interviewee availability matrix
                V_E_dense[t,e] = 1
            end
        end
    end
end

# subdivide the availability matrix into smaller pieces
V_E = subdivideAvailabilityMatrix(transpose(V_E_dense))

# transpose the availability matrix
V_E = transpose(V_E);


In [90]:
# Read in the interviewee data
raw = readtable("Interviewee Information.csv");

# describe the formatting of the data table
nameCol = 1
emailCol = 2
com1Col = 3
com2Col = 4
friCol = 5
satCol = 6
sunCol = 7

# extract the names of the interviewees
inter_names = raw[:,nameCol]
n_I = length(inter_names) #the number of interviewees

# extract the first committee each interviewee is interviewing for
## define the list of possible committees
committees = ["Brunch", "Operations", "Coffee Hour", "Communities", "Events"]

## initialize the list of committee names corresponding to the interviewees
inter_com_names = fill("",n_I)
for i in 1:n_I
    
    # look for a committee
    committeeFound = false
    for committee in committees
        if contains(raw[i,com1Col],committee)
            # set the committee found flag
            committeeFound = true
            
            # set the committee name for this interviewee
            inter_com_names[i] = committee
        end
    end
    
    #make sure that a committee was found
    if committeeFound == false
        error("No committee was found for interviewee $i in the string $raw[i,com1Col].") 
    end
end


# initialize the committee code matrix for the interviewees
M_I = zeros(Integer,n_I,2)

# convert the committee names into committee codes
for i = 1:n_I
    M_I[i,1] = lookup_com_codes[inter_com_names[i]]
end

# extract the second committee each interviewee is interviewing for
for i in 1:n_I
    
    # look for a committee
    committeeFound = false
    for committee in committees
        if contains(raw[i,com2Col],committee)
            # set the committee found flag
            committeeFound = true
            
            # set the committee name for this interviewee
            inter_com_names[i] = committee
        end
    end
    
    #make sure that a committee was found
    if committeeFound == false
        error("No committee was found for interviewee $i in the string $raw[i,com2Col].") 
    end
end

# convert the committee names into committee codes
for i = 1:n_I
    M_I[i,2] = lookup_com_codes[inter_com_names[i]]
end

# extract the subset of the interviewees that are applying for the Tech committee
I_E = Int64[];
for i = 1:n_I
    if contains(raw[i,com1Col],"Tech") || contains(raw[i,com2Col],"Tech")
        I_E = push!(I_E, i); 
    end
end

# Extract the email addresses of the interviewees
emails = raw[1:n_I,emailCol];

# Read in the interviewee availability
## initialize the availability matrix
V_I_dense = zeros(Integer,n_T_coarse,n_I)

## loop over the interviewees
for i = 1:n_I
    # extract the friday availability
    timesString = raw[i,friCol]
    for time in friTimes
        if contains(timesString,time) # this interviewee indicated that they are available during the current time
            # map the current time string to the corresponding (coarse) time index
            t = mapDayAndTimeToCoarseIndex("Friday",time)
            
            # populate the corresponding entry of the interviewee availability matrix
            V_I_dense[t,i] = 1
        end
    end
    
    # extract the saturday availability
    timesString = raw[i,satCol]
    for time in satTimes
        if contains(timesString,time) # this interviewee indicated that they are available during the current time
            # map the current time string to the corresponding (coarse) time index
            t = mapDayAndTimeToCoarseIndex("Saturday",time)
            
            # populate the corresponding entry of the interviewee availability matrix
            V_I_dense[t,i] = 1
        end
    end
    
    # extract the sunday availability
    timesString = raw[i,sunCol]
    for time in sunTimes
        if contains(timesString,time) # this interviewee indicated that they are available during the current time
            # map the current time string to the corresponding (coarse) time index
            t = mapDayAndTimeToCoarseIndex("Sunday",time)
            
            # populate the corresponding entry of the interviewee availability matrix
            V_I_dense[t,i] = 1
        end
    end
    
end # end loop over interviewees

# subdivide the dense availability matrix into smaller pieces
V_I = subdivideAvailabilityMatrix(transpose(V_I_dense));
V_I = transpose(V_I);

In [91]:
# Define the sets
T = 1:n_T
I = 1:n_I
A = 1:n_A
O = 1:n_O
C = 1:n_C
E = 1:n_E;


# Define and Solve the Optimization Problem
## Initialize the Model and Define the Decision Variables
The first line of this cell can be modified to set the maximum time you'll allow the solver to work on improving the schedule.
Obviously, the longer you let it run, the better the schedule you'll get.
But there are diminishing returns.
Your time is limited, and the solver can find a really good (if not optimal) schedule in a relatively short time.

In [92]:
# set the maximum solve time in minutes
maxSolveTime = 1; #minutes (recommendation: 6 minutes)

# DEFINE THE OPTIMIZATION PROBLEM
using JuMP
# using Cbc
# m = Model(solver=CbcSolver(Sec=maxSolveTime*60))

using Gurobi
m = Model(solver=GurobiSolver(TimeLimit=maxSolveTime*60))

# Define the decision variables
@variable(m, x_I[T,I,1:2], Bin)
@variable(m, x_A[T,A], Bin)
@variable(m, x_O[T,O], Bin)
@variable(m, x_C[T,C], Bin)
@variable(m, x_E[T,E], Bin)
@variable(m, y[T,I], Bin)
@variable(m, z[T], Bin)

## Define the Constraints

In [93]:
# Define the constraints
# constraint group 1
for t in T
    for i in I
        @constraint(m, x_I[t,i,1] <= y[t,i])
        @constraint(m, x_I[t,i,2] <= y[t,i])
        @constraint(m, y[t,i] <= x_I[t,i,1] + x_I[t,i,2])
    end
end

# constraint group 2
for i in I
    @constraint(m, sum(x_I[:,i,1]) == 1) 
end

# constraint group 3
for i in I
    if M_I[i,2] > 0
        @constraint(m, sum(x_I[:,i,2]) == 1) 
    end
end

# constraint group 4
for t in T
    @constraint(m, sum(y[t,:]) <= 1) 
end

# constraint group 5
for t in T
    for a in A
        @constraint(m, x_A[t,a] <= V_A[t,a])
    end
end

# constraint group 6
for t in T
    for c in C
        @constraint(m, x_C[t,c] <= V_C[t,c])
    end
end

# constraint group 7
for t in T
    for a in A
        @constraint(m, x_A[t,a] <= z[t] + sum(x_I[t,i,j] for i in I, j = 1:2 if M_A[a] == M_I[i,j]) )
    end
end

# constraint group 8
for t in T
    for c in C
        @constraint(m, x_C[t,c] <= sum(x_I[t,i,j] for i in I, j = 1:2 if M_C[c] == M_I[i,j]) )
    end
end
    
# constraint group 9
for t in T
    for i in I
        @constraint(m, y[t,i] <= sum(x_A[t,:]));
    end
end
    
#constraint group 10
ops_code = lookup_com_codes["Operations"];
for i in I_E
    for j = 1:2
        if M_I[i,j] == ops_code
            for t in T
                @constraint(m, x_I[t,i,j] <= sum(x_E[t,:])) 
            end
        end
    end
end
    
# constraint group 11
for t in T
    for e in E
        @constraint(m, x_E[t,e] <= V_E[t,e])
    end
end
    
# constraint group 12
for t in T
    for e in E
        @constraint(m, x_E[t,e] <= sum(x_I[t,i,j] for i = I_E, j = 1:2 if (M_I[i,j] == ops_code)))
    end
end
                

# constraint group 13
for t in T
    @constraint(m, sum(x_A[t,:]) <= 2 - z[t]) 
end
                        
# constraint group 14
for t = 1:(n_T - 5)
    for a in A
        @constraint(m, sum(x_A[tau ,a] for tau = t:(t+5)) <= 5)
    end
end
                            
# constraint group 15
for t = 1:(n_T - 5)
    for c in C
        @constraint(m, sum(x_C[tau ,c] for tau = t:(t+5)) <= 5)
    end
end
                                
# constraint group 16
for t in T
    for o in O
        @constraint(m, x_O[t,o] <= V_O[t,o])
    end
end
                                
# constraint group 17
for t in T
    for o in O
        @constraint(m, x_O[t,o] <= sum(x_I[t,i,j] for i in I, j = 1:2 if M_O[o] == M_I[i,j]) )
    end
end
                                        
# constraint group 18
for t = 1:(n_T - 5)
    for o in O
        @constraint(m, sum(x_O[tau ,o] for tau = t:(t+5)) <= 5)
    end
end
                                            
# constraint group 19
for t in T
    @constraint(m, sum(x_E[t,:]) <= 1) 
end

## Define the Objective Function
The objective function has several weighting coefficients, $w_m, w_A, w_O, w_C, w_I$, and $w_z$.
Each of these coefficients corresponds to a particular objective.

| Weighting Coefficient | Suggested Value | Objective |
|:---------------------:|:---------------:|-----------|
| $w_m$                 | 1               | Schedule as few meetings as possible.     |
| $w_A$                 | 0.2               | Each AHEC member should attend the interview when someone is interviewing for their committee.     |
| $w_O$                 | 0.1               | Each old AHEC member should attend the interview when someone is interviewing for their committee.     |
| $w_C$                 | 0.2               |Each committee chair should attend the interview when someone is interviewing for their committee.     |
| $w_I$                 | 0.5               |Only schedule interviewees to attend interviews when the interviewee is available.  |
| $w_z$                 | 1               |Interviewees can only interview for a committee when the corresponding AHEC member is present. |

You can set the values of these coefficients in the code below. 
Each coefficient most be nonnegative (i.e., $w_i \geq 0$).
**The greater the value of the coefficient, the harder the code will try to accomplish the corresponding objective.** That being said, if you double each coefficient, it will make no difference to the code, because it is really the *ratios* between them that matter.

Manipulating these weighting coefficients will very likely effect the summary statistics and schedule produced by the code, but small changes in the weights may have no effect.

In [94]:
# Define the objective
# specify the penalty weights
w_m = 1;
w_A = 0.2; #reward for an AHEC member attending an interview for their committee
w_O = 0.1; #reward for an AHEC member attending an interview for their committee
w_C = 0.2; #reward for a Committee Chair attending an interview for their committee.
w_I = 0.5; #penalty for scheduling an interviewee to meet when they are not available.
w_z = 1; #penalty for making an exception

P1 = w_m*sum(sum(y))
P2 = -w_A*sum(sum(x_A))
P3 = -w_C*sum(sum(x_C))
P4 = -w_O*sum(sum(x_O))

# initialize P5
P5 = 0
for t in T, i in I
    P5 = P5 + w_I*(1 - V_I[t,i])*y[t,i]
end

P6 = w_z*sum(z)

@objective(m, Min, P1 + P2 + P3 + P4 + P5 + P6);

## Solve the Optimization Problem
The code will spend most of its time here.
You can manipulate the amount of time it spends using the maxSolveTime parameter you defined when you initialized the model.
Since you are limiting the maximum solve time, when this cell finishes, it will say: 

    "WARNING: Not solved to optimality, status: UserLimit".

That's OK.
This is expected behavior.

In [95]:
# Solve the Optimization Problem
status = solve(m)
println(status)

Optimize a model with 26516 rows, 17304 columns and 124404 nonzeros
Variable types: 0 continuous, 17304 integer (17304 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 5948 rows and 1818 columns
Presolve time: 0.21s
Presolved: 20568 rows, 15486 columns, 82083 nonzeros
Variable types: 0 continuous, 15486 integer (15486 binary)

Root relaxation: objective 2.260000e+01, 12961 iterations, 0.67 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   22.60000    0  166          -   22.60000      -     -    1s
H    0     0                      55.9000000   22.60000  59.6%     -    1s
H    0     0                      30.5000000   22.60000  25.9%     -    1s
     0     0   22.78333    0  349   30.50000   22.78333  25.3%     -    2s
H    0     

# Check the Results
Since we are limiting the solve time, it is necessary to check that the solver actually found a feasible solution (satisfying all of our constraints) in the time provided.
This check is performed by the following cell.
If the output is "The schedule is feasible.  Proceed.", the solution was indeed feasible.
On the other hand, if the output is "Infeasible", the solution does not satisfy one or more constraints.
We need to allow the solver to try again, and give it more time to work on the problem.

In [96]:
# Extract and round the answer to account for numerical imprecisions
## round the results to the nearest integers
x_I_star = zeros(Int,n_T,n_I,2)
for t in T, i in I, j = 1:2
    x_I_star[t,i,j] = round(Int,getvalue(x_I[t,i,j]));
end

x_A_star = zeros(Int,n_T,n_A)
for t in T, a in A
    x_A_star[t,a] = round(Int,getvalue(x_A[t,a]));
end

x_O_star = zeros(Int,n_T,n_O)
for t in T, o in O
    x_O_star[t,o] = round(Int,getvalue(x_O[t,o]));
end

x_C_star = zeros(Int,n_T,n_C)
for t in T, c in C
    x_C_star[t,c] = round(Int,getvalue(x_C[t,c]));
end

x_E_star = zeros(Int,n_T,n_E)
for t in T, e in E
    x_E_star[t,e] = round(Int,getvalue(x_E[t,e]));
end

y_star = zeros(Int,n_T,n_I)
for t in T, i in I
    y_star[t,i] = round(Int,getvalue(y[t,i]));
end

z_star = zeros(Int,n_T)
for t in T
    z_star[t] = round(Int,getvalue(z[t]));
end

# Check the feasibiliy of the solution
## fix the decision  variables
for t in T, i in I, j in 1:2
    @constraint(m, x_I[t,i,j] == x_I_star[t,i,j])
end

for t in T, a in A
    @constraint(m, x_A[t,a] == x_A_star[t,a]) 
end

for t in T, o in O
    @constraint(m, x_O[t,o] == x_O_star[t,o]) 
end

for t in T, c in C
    @constraint(m, x_C[t,c] == x_C_star[t,c]) 
end

for t in T, e in E
    @constraint(m, x_E[t,e] == x_E_star[t,e]) 
end

for t in T, i in I
    @constraint(m, y[t,i] == y_star[t,i]) 
end

for t in T
    @constraint(m, z[t] == z_star[t])
end

## modify the objective function to make it a feasibility problem
@objective(m,Min,0)

## call the solver again
status = solve(m)
if status == :Optimal
    println("The schedule is feasible.  Proceed.")
elseif status == :Infeasible
    println("The schedule violates one or more constraints.  Start over, giving the solver more time to work on the problem.")
else
    println("Unexpected status: $status.  Talk to Garrett.")
end

Optimize a model with 43820 rows, 17304 columns and 141708 nonzeros
Variable types: 0 continuous, 17304 integer (17304 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 0

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
The schedule is feasible.  Proceed.


# Export the Results
The following two cells construct a table showing the schedule suggested by the solver, and then outputs this table as a .csv file to JuliaBox under the name "Composite Schedule.csv".


In [97]:
## export the composite schedule
### initialize the arrays
period = zeros(Int,n_T) #fill("",n_T)
interviewee = fill("",n_T)
commChair1 = fill("",n_T)
commChair2 = fill("",n_T)
ahec1 = fill("",n_T)
ahec2 = fill("",n_T)
oldahec1 = fill("",n_T)
oldahec2 = fill("",n_T)
comm1 = fill("",n_T)
comm2 = fill("",n_T)
tech = fill("",n_T)
email = fill("",n_T)
days = fill("",n_T)
times = fill("",n_T)

### fill in the arrays
for t in T
    # fill in the period
    period[t] = t
    
    # look for an interviewee
    intervieweeFound = false
    for i in I
        if y_star[t,i] == 1 # an interviewee was found
            # check that this is the first interviewee for this period
            if intervieweeFound == false # this was the first interviewee
                # fill in the array with the interviewee's name
                interviewee[t] = inter_names[i]

                # indicate that an interviewee has been found
                intervieweeFound = true

                # fill in the committees
                if x_I_star[t,i,1] == 1 
                    comm1[t] = lookup_com_names[M_I[i,1]];
                end
                if x_I_star[t,i,2] == 1 && x_I_star[t,i,1] == 1
                    comm2[t] = lookup_com_names[M_I[i,2]];
                end
                if x_I_star[t,i,2] == 1 && x_I_star[t,i,1] == 0
                    comm1[t] = lookup_com_names[M_I[i,2]];
                end
                
                # add the "tech" designation, if necessary
                if i in I_E
                    if comm1[t] == "Operations"
                        comm1[t] = "Operations (Tech)"
                    elseif comm2[t] == "Operations"
                        comm2[t] = "Operations (Tech)"
                    end
                end
                
                # fill in the email
                email[t] = emails[i]
            else
                error("Two interviewees were scheduled for the same period.") 
            end
        end
    end
    
    # look for the AHEC reps
    repsFound = 0;
    for a in A
        if x_A_star[t,a] == 1 # an AHEC rep has been assigned
            # check that there haven't been too many reps assigned to this interview
            if repsFound < 2 # there have not been too many assigned
               # increment the count of reps found
               repsFound = repsFound + 1;
               
               # fill in the cell in the table
               if repsFound == 1
                    ahec1[t] = AHEC_names[a];
               elseif repsFound == 2
                    ahec2[t] = AHEC_names[a];
               end
           else
               error("More than two AHEC reps were scheduled for the same period.")
           end
       end
    end
    
    # look for the old AHEC reps
    repsFound = 0;
    for o in O
        if x_O_star[t,o] == 1 # an old AHEC rep has been assigned
            # check that there haven't been too many reps assigned to this interview
            if repsFound < 2 # there have not been too many assigned
               # increment the count of reps found
               repsFound = repsFound + 1;
               
               # fill in the cell in the table
               if repsFound == 1
                    oldahec1[t] = oldAHEC_names[o];
               elseif repsFound == 2
                    oldahec2[t] = oldAHEC_names[o];
               end
           else
                error("More than two old AHEC reps were scheduled for the same period.")
           end
       end
    end
    
    # look for the Committee Chair reps
    repsFound = 0;
    for c = C
        if x_C_star[t,c] == 1
            # check that there haven't been too many reps assigned to this interview
            if repsFound < 2 # there haven't been too many reps assigned
               # increment the count of reps found
               repsFound = repsFound + 1
               
               # fill in the cell in the table
               if repsFound == 1
                    commChair1[t] = chair_names[c]
               elseif repsFound == 2
                    commChair2[t] = chair_names[c]
               end
           else
#                error("More than two committee chair reps were scheduled for the same period.")
           end
       end
    end
    
    # look for the Tech Committee reps
    repsFound = 0;
    for e = E
        if x_E_star[t,e] == 1
            # increment the count of reps found
            repsFound = repsFound + 1;

            # fill in the cell in the table
            if repsFound == 1
                tech[t] = tech_names[e];
            else
                tech[t] = string(tech[t], ", ", tech_names[e])  #lengthen the list of tech names
            end
       end
    end
    
    # construct the time string
    ## map t to the corresponding coarse time index
    t_coarse = div(t-1,3) + 1
    
    ## map the coarse index to a particular day and time string
    if t_coarse <= length(friTimes)
        day = "Friday"
        hour = friTimes[t_coarse]
    elseif t_coarse <= length(friTimes) + length(satTimes)
        day = "Saturday"
        hour = satTimes[t_coarse - length(friTimes)]
    elseif t_coarse <= length(friTimes) + length(satTimes) + length(sunTimes)
        day = "Sunday"
        hour = sunTimes[t_coarse - length(friTimes) - length(satTimes)]
    else
        error("The coarse time index $t_coarse has an unexpected value.")
    end
    
    ## add in the minutes
    ### extract the various parts of the hour string
    dashInd = searchindex(hour,"-")
    apInd = searchindex(hour,"m") - 1
    startHour = hour[1:(dashInd-1)]
    endHour = hour[(dashInd+1):(apInd-1)]
    apm = hour[apInd:end]
    
    ### calculate the minutes offset    
    minutesOffset = rem(t-1,3)
    if minutesOffset == 0
        #specify the minutes
        minutes1 = ":00"
        minutes2 = ":20"
        
        #adjust the am/pm strings as necessary
        if endHour == "12"
            apm1 = "am"
            apm2 = "am"
        else
            apm1 = apm
            apm2 = apm
        end
        
        #piece togther the time string
        time = "$startHour$minutes1 $apm1-$startHour$minutes2 $apm2"
    elseif minutesOffset == 1
        #specify the minutes
        minutes1 = ":20"
        minutes2 = ":40"
           
        #adjust the am/pm strings as necessary
        if endHour == "12"
            apm1 = "am"
            apm2 = "am"
        else
            apm1 = apm
            apm2 = apm
        end
        
        #piece togther the time string
        time = "$startHour$minutes1 $apm1-$startHour$minutes2 $apm2"
    elseif minutesOffset == 2
        #specify the minutes
        minutes1 = ":40"
        minutes2 = ":00"
        
        #adjust the am/pm strings as necessary
        if endHour == "12"
            apm1 = "am"
            apm2 = "pm"
        else
            apm1 = apm
            apm2 = apm
        end
        
        time = "$startHour$minutes1 $apm1 -$endHour$minutes2 $apm2"
    else
        error("The value of minutes offset = $minutesOffset has an unexpected value.") 
    end

    ## fill in the entries of the day and hour columns
    days[t] = day
    times[t] = time
         
end
    
# put the table together
df = DataFrame(Day = days, Time = times, Interviewee = interviewee, Email = email, Committee_1 = comm1, Committee_2 = comm2, AHEC_1 = ahec1, AHEC_2 = ahec2, Committee_Chair_1 = commChair1, Committee_Chair_2 = commChair2, Tech = tech, Old_AHEC_1 = oldahec1, Old_AHEC_2 = oldahec2 )

,Day,Time,Interviewee,Email,Committee_1,Committee_2,AHEC_1,AHEC_2,Committee_Chair_1,Committee_Chair_2,Tech,Old_AHEC_1,Old_AHEC_2
1,Friday,3:00 pm-3:20 pm,,,,,,,,,,,
2,Friday,3:20 pm-3:40 pm,,,,,,,,,,,
3,Friday,3:40 pm -4:00 pm,,,,,,,,,,,
4,Friday,4:00 pm-4:20 pm,,,,,,,,,,,
5,Friday,4:20 pm-4:40 pm,,,,,,,,,,,
6,Friday,4:40 pm -5:00 pm,,,,,,,,,,,
7,Friday,5:00 pm-5:20 pm,,,,,,,,,,,
8,Friday,5:20 pm-5:40 pm,,,,,,,,,,,
9,Friday,5:40 pm -6:00 pm,,,,,,,,,,,
10,Friday,6:00 pm-6:20 pm,,,,,,,,,,,


In [98]:
# write the results to Excel
writetable("Composite Schedule.csv",df)

# Display Statistics
This code prints some statistics that give you a sense of the quality of the schedule generated.

In [99]:
# Report statistics
## report the total number of meetings
meetings = sum(sum(y_star))
println("The total number of meetings is: $meetings")

## report the percentage of meetings schedule when the interviewees are available
availableMeetings = sum(sum(V_I.*y_star))/sum(sum(y_star))
println("The fraction of the meetings for which the interviewees are available is: $availableMeetings")

## report the number of meetings AHEC members must attend
AHECmeetings = sum(sum(x_A_star))
println("The number of times that an AHEC member must go to a meeting is: $AHECmeetings")

# ## report the number of exceptions to rules
# AHECexceptions = sum(sum(z_A_star))
# println("The number of AHEC attendance exceptions is: $AHECexceptions")
# commChairexceptions = sum(sum(z_C_star))
# println("The number of committee chair attendance exceptions is: $commChairexceptions")
AHECexceptions = sum(z_star)
println("The number of AHEC attendance exceptions is: $AHECexceptions")


The total number of meetings is: 63
The fraction of the meetings for which the interviewees are available is: 1.0
The number of times that an AHEC member must go to a meeting is: 85
The number of AHEC attendance exceptions is: 0
